In [1]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!env CUDA_DEVICE_ORDER=PCI_BUS_ID
!env CUDA_VISIBLE_DEVICES=4

LMOD_DIR=/usr/share/lmod/lmod/libexec/
TMUX=/tmp/tmux-22086/default,1030980,0
SSH_CLIENT=103.107.4.31 63325 22
USER=m13518040
LMOD_COLORIZE=yes
MPLBACKEND=module://matplotlib_inline.backend_inline
LMOD_PKG=/usr/share/lmod/lmod
LD_LIBRARY_PATH=/usr/local/cuda/lib:/usr/local/cuda/lib
HOME=/home/m13518040
MOTD_SHOWN=pam
OLDPWD=/raid/data/m13518040
SSH_TTY=/dev/pts/2
PAGER=cat
LMOD_sys=Linux
LOGNAME=m13518040
TERM=xterm-color
PATH=/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
LMOD_FULL_SETTARG_SUPPORT=no
LMOD_PREPEND_BLOCK=normal
LANG=en_US.UTF-8
MODULEPATH_ROOT=/sw/modules
SHELL=/bin/sh
LMOD_CMD=/usr/share/lmod/lmod/libexec/lmod
BASH_ENV=/usr/share/lmod/lmod/init/bash
GIT_PAGER=cat
PWD=/raid/data/m13518040/tugas-akhir-repository
CLICOLOR=1
SSH_CONNECTION=103.107.4.31 63325 167.205.35.2

In [2]:
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, load_word_index
import time
from nltk.tokenize import word_tokenize
import torch
import torch.nn.functional as F
# import jax.numpy as jnp
# from jax import random

import math
# from .attack_helper import get_synonyms, codemix_perturbation, synonym_replacement, swap_minimum_importance_words, trans_dict

import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

from icecream import ic


import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
# nltk.download('omw-1.4')

import copy
from deep_translator import GoogleTranslator
import numpy as np
import random
# from utils.utils_semantic_use import USE
from utils.utils_init_dataset import set_seed, load_dataset_loader
from sklearn.metrics import accuracy_score

import torch
device = torch.device("cuda")

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import tensorflow_hub as hub
import os

import jax.numpy as jnp

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# tf.config.run_functions_eagerly(True)

In [3]:
# !pip install jax
# !pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.

In [4]:
class USE(object):
    def __init__(self):
        import tensorflow_hub as hub
        super(USE, self).__init__()
        module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
        self.embed = hub.load(module_url)

    def semantic_sim(self, sentA : str, sentB : str) -> float:
        """
        Args:
            sentA: The first sentence.
            sentB: The second sentence.

        Returns:
            Cosine distance between two sentences.
        
        """
        if sentA.lower() == sentB.lower():
            return 1.000
        ret = np.array(self.embed([sentA, sentB]).numpy())
        score = ret[0].dot(ret[1]) / (np.linalg.norm(ret[0]) * np.linalg.norm(ret[1]))
        if score>1:
            return 1.000
        return score

    def after_attack(self, input, adversarial_sample):
        if adversarial_sample is not None:
            return self.calc_score(input["x"], adversarial_sample)
# tf.config.run_functions_eagerly(True)
# USE()

In [5]:
# class USE(object):
#     def __init__(self, module_url=None):
#         super(USE, self).__init__()
#         module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
#         self.embed = hub.load(module_url)
        
#         def embed(input):
#             return model(input)
        
#         # config = tf.compat.v1.ConfigProto()
#         # config.gpu_options.allow_growth = True
#         # self.sess = tf.compat.v1.Session(config=config)
#         # self.sess.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])

#     def semantic_sim(self, sents1, sents2):
#         # with tf.Session() as session:
#         #     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#         #     with tf.device('GPU'):
#         #         message_embeddings_ = session.run(self.embed([sents1, sents2]))
        
#         if sents1.lower() == sents2.lower():
#             return 1.000
        
#         message_embeddings_ = self.embed([sents1, sents2])
#         # message_embeddings_ = message_embeddings_.eval(session=self.sess)
#         corr = np.tensordot(message_embeddings_, message_embeddings_, axes=(-1,-1))
        
#         if corr[0][1] > 1:
#             return 1.000
#         return corr[0][1]

In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def get_synonyms(word):
    if word in stop_words_set:
        return [word]

    word_lemmas = wordnet.lemmas(word, lang="ind")
    
    hypernyms = []
    for lem in word_lemmas:
        hypernyms.append(lem.synset().hypernyms())

    if not any(hypernyms):
        return [word]
    
    lemma_corp = []
    
    for hypernym in hypernyms:
        if(len(hypernym) < 1):
            continue
        else:
            lemma_corp.append(hypernym[0].lemmas(lang="ind"))
            
    lemmas = set()
    for list_lemmas in lemma_corp:
        if(len(list_lemmas) < 1):
            lemmas.add(word)
        else:
            for l in list_lemmas:
                lemmas.add(l.name())
    
    clean_synonyms = set()
    for syn in lemmas.copy():
        synonym = syn.replace("_", " ").replace("-", " ").lower()
        synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
        clean_synonyms.add(synonym) 
    if word in clean_synonyms:
        clean_synonyms.remove(word)
    
    if len(list(clean_synonyms)) < 1:
        return [word]
    else:
        return list(clean_synonyms)

def codemix_perturbation(words, target_lang, words_perturb):
    """
    'su': 'sundanese'
    'jw': 'javanese'
    'ms': 'malay'
    'en': 'english'
    """
    translator = GoogleTranslator(source="id", target=target_lang)
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
    elif len(new_wp) == 0:
        return ' '.join(words), {}
        
    new_wp_trans = dict(zip(new_wp, translator.translate_batch(new_wp)))
    
    supported_langs = ["su", "jw", "ms", "en", "fr", "it"]
    
    if target_lang not in supported_langs:
        raise ValueError('Language Unavailable')
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word] if word == perturb_word and word.isalpha() else word for word in new_words]

    return ' '.join(new_words), new_wp_trans

def synonym_replacement(words, words_perturb):    
    # new_words = words.copy()
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, get_synonyms(new_wp[0])[0]))
    elif len(new_wp) == 0:
        return ' '.join(words), {}
    
    wp_trans=[]
    for wp in new_wp:
        wp_trans.append(get_synonyms(wp)[0])
    
    new_wp_trans = dict(zip(new_wp, wp_trans))
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word] if word == perturb_word and word.isalpha() else word for word in new_words]
    
    return ' '.join(new_words), new_wp_trans

# fungsi untuk mencari kandidat lain ketika sebuah kandidat perturbasi kurang dari sim_score_threshold
def swap_minimum_importance_words(words_perturb, top_importance_words):
    print("pointer")
    def get_minimums(word_tups):
        arr = []
        for wt in word_tups:
            if wt[2] == min(top_importance_words, key = lambda t: t[2])[2]:
                arr.append(wt)
        return arr
    
    # get list of words with minimum importance score
    minimum_import = get_minimums(top_importance_words)
    unlisted = list(set(words_perturb).symmetric_difference(set(top_importance_words)))
    
    ic(unlisted)
    
    len_wp = len(top_importance_words)
    len_ul = len(unlisted)
    
    res = []
    for i in range(len_wp):
        if top_importance_words[i] in minimum_import:
            temp_wp = list(copy.deepcopy(top_importance_words))
            if len(temp_wp) == 0:
                break
            
            swapped_wp = np.array([(temp_wp) for i in range(len_ul)])
            
            ic(swapped_wp)
            for j in range(len(swapped_wp)):
                temp_sm = np.vstack((swapped_wp[j], tuple(unlisted[j])))
                
                res.append(temp_sm.tolist())
            temp_wp.pop(i)
                
    return res

def trans_dict(wordlist, perturbation_technique, lang=None):
    translated_wordlist = {}
    # ic(wordlist)
    if perturbation_technique == "codemixing":
        translator = GoogleTranslator(source="id", target=lang)
        
        for word in wordlist:
            if word.isalpha():
                translated_wordlist[word] = translator.translate(word)
            else:
                translated_wordlist[word] = word
    else:
        for word in wordlist:
            translated_wordlist[word] = get_synonyms(word)[0]
            
    return translated_wordlist

In [15]:
def logit_prob(text_ls, predictor, tokenizer, batch=False):
    with torch.inference_mode():
        if(batch==True):
            subwords = tokenizer.batch_encode_plus(text_ls, return_tensors="pt", padding=True)['input_ids'].to(finetuned_model.device)
            logits = finetuned_model(subwords).logits
            # ic(logits)
            orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
        else:
            subwords = tokenizer.encode(text_ls)
            subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)

            logits = predictor(subwords)[0]
            orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()

            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
    
    return orig_label, orig_probs, orig_prob

    # subwords = tokenizer.encode(text_ls)
    # subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)
    # logits = predictor(subwords)[0]
    # orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    # orig_probs = F.softmax(logits, dim=-1).squeeze()
    # orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()

def attack(text_ls,
           true_label,
           predictor,
           tokenizer,
           att_ratio,
           perturbation_technique,
           lang_codemix=None,
           sim_predictor=None,
           sim_score_threshold=0,
           sim_score_window=15,
           batch_size=32, 
           import_score_threshold=-1.):
    
    start_time = time.time()
    label_dict = {
        'positive': 0, 
        'neutral': 1, 
        'negative': 2}
    
    original_text = text_ls
    splitted_text = text_ls.split()
    
    orig_label, orig_probs, orig_prob = logit_prob(text_ls, predictor, tokenizer)
    
    if len(splitted_text) > sim_score_window:
        sim_score_threshold = 0.1  
    #     SEK SALAAHHHHHH
    if true_label != orig_label:
        running_time = round(time.time() - start_time, 2)
        # perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        return original_text, 1.000, orig_label, orig_probs, orig_label, orig_probs,'', running_time
    else:
        text_ls = splitted_text
        text_ls = [word for word in text_ls if word.isalnum()]
        len_text = len(text_ls)
        # ic(len_text)
        # ic(original_text)
        half_sim_score_window = (sim_score_window - 1) // 2
        # num_queries = 1
        
        leave_1_texts = [' '.join(text_ls[:ii] + [tokenizer.mask_token] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]
        # ic(text_ls)
        # ic(leave_1_texts)
                
        leave_1_probs = []
        leave_1_probs_argmax = []
        # num_queries += len(leave_1_texts)
        
# torch.inference mode
        # start_time_logit = time.time()
        if len(text_ls) <=1:
            leave_1_probs_argmax, leave_1_probs, _ = logit_prob(leave_1_texts, predictor, tokenizer)
            leave_1_probs_argmax = [leave_1_probs_argmax]
            leave_1_probs = [leave_1_probs.detach().cpu().numpy()]
            # ic(leave_1_probs)
            leave_1_probs = torch.tensor(np.array(leave_1_probs)).to("cuda:0")
        else:
            leave_1_probs_argmax, leave_1_probs, _ = logit_prob(leave_1_texts, predictor, tokenizer,batch=True)
            leave_1_probs = torch.tensor(np.array(leave_1_probs.detach().cpu().numpy())).to("cuda:0")
        
        # end_time_logit = round(time.time() - start_time_logit, 4)
        # ic(end_time_logit)
        
        # ic(leave_1_probs)
        
        # start_time_arrops = time.time()
        # leave_1_probs = torch.tensor(np.array(leave_1_probs)).to("cuda:0")
        # leave_1_probs = torch.tensor(np.array(leave_1_probs.detach().cpu().numpy())).to("cuda:0")
        # ic(leave_1_probs)
        
        orig_prob_extended=np.empty(len_text)
        orig_prob_extended.fill(orig_prob)
        # ic(orig_prob_extended)
        orig_prob_extended = torch.tensor(orig_prob_extended).to("cuda:0")
        
        # ic(leave_1_probs)
        arr1 = orig_prob_extended - leave_1_probs[:,orig_label] + float(leave_1_probs_argmax != orig_label)
        arr2 = (leave_1_probs.max(dim=-1)[0].to("cuda:0") - orig_probs[leave_1_probs_argmax].to("cuda:0"))
        
        import_scores = arr1*arr2
        import_scores = [im * -1 for im in import_scores]
        
        words_perturb = []
        for idx, score in sorted(enumerate(import_scores), key=lambda x: x[1], reverse=True):
            try:
                if score > import_score_threshold and text_ls[idx] not in stop_words_set:
                    words_perturb.append((idx, text_ls[idx], score.item()))
            except Exception as e:
                print(e)
                print(idx, len(text_ls), import_scores.shape, text_ls, len(leave_1_texts))
        
        num_perturbation = math.floor(len(words_perturb)*att_ratio)
        
        # end_time_arrops = round(time.time() - start_time_arrops, 4)
        # ic(end_time_arrops)
        
        if num_perturbation < 1:
            num_perturbation = 1
        
#       top words perturb berisi list kata terpenting yang tidak akan diswitch ketika first_codemix_sim_score < sim_score_threshold
        top_words_perturb = words_perturb[:num_perturbation]
        trans_word = [twp[1] for twp in top_words_perturb]
        
        # running_time_a = round(time.time() - start_time, 4)
        # ic(running_time_a)
        # ic(top_words_perturb)
        # start_time_pert = time.time()
        if perturbation_technique == "codemixing":
            perturbed_text,translated_words = codemix_perturbation(text_ls, lang_codemix, top_words_perturb)
        elif perturbation_technique == "synonym_replacement":
            perturbed_text,translated_words = synonym_replacement(text_ls, top_words_perturb)
        # end_time_pert = round(time.time() - start_time_pert, 4)
        # ic(end_time_pert)
        
        
        # start_time_use = time.time()
        first_perturbation_sim_score = sim_predictor.semantic_sim(original_text, perturbed_text)
        # end_time_use = round(time.time() - start_time_use, 4)
        
        # ic(end_time_use)
        
        # ic(first_perturbation_sim_score)
        
        words_perturb_candidates = []
        if first_perturbation_sim_score < sim_score_threshold:
            words_perturb_candidates.append(top_words_perturb)
            swapped = swap_minimum_importance_words(words_perturb, top_words_perturb)
            for s in swapped:
                words_perturb_candidates.append(s)

            words_perturb_candidates = [[tuple(w) for w in wpc] for wpc in words_perturb_candidates]

            candidate_comparison = {}
            for wpc in words_perturb_candidates:
                
                if perturbation_technique == "codemixing":
#                     bisa dicache, gaperlu request satu2
                    perturbed_candidate,translated_words = codemix_perturbation(text_ls, lang_codemix, wpc)
                elif perturbation_technique == "synonym_replacement":
                    perturbed_candidate,translated_words = synonym_replacement(text_ls, wpc)
                    
                perturbed_candidate_sim_score = sim_predictor.semantic_sim(original_text, perturbed_candidate)
                
                candidate_comparison[perturbed_candidate] = (perturbed_candidate_sim_score, wpc[-1][-1], wpc)

            sorted_candidate_comparison = sorted(candidate_comparison.keys(), key=lambda x: (float(candidate_comparison[x][0]), float(candidate_comparison[x][1])), reverse=True)

            perturbed_text = sorted_candidate_comparison[0]
            
            top_words_perturb = candidate_comparison[sorted_candidate_comparison[0]][-1]
            trans_word = [twp[1] for twp in top_words_perturb]
            
        perturbed_semantic_sim = sim_predictor.semantic_sim(original_text, perturbed_text)
        if perturbed_semantic_sim < sim_score_threshold:
            perturbed_text = original_text
            perturbed_semantic_sim = 1.000
        
        perturbed_label, perturbed_probs, perturbed_prob = logit_prob(perturbed_text, predictor, tokenizer)
        
        
        # translated_words = trans_dict(trans_word, perturbation_technique, lang_codemix)
        
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        perturbed_probs = np.round(perturbed_probs.detach().cpu().numpy().tolist(),4)
        
        running_time = round(time.time() - start_time, 4)
        # ic(running_time)
        
        # ic(perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time)
        return perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time

In [8]:
model_target="IndoBERT"
downstream_task="sentiment"
attack_strategy="adversarial"
# finetune_epoch=1
num_sample=50
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.4
perturb_lang="en"
seed=26092020
dataset="valid"

set_seed(seed)
use = USE()

2022-04-17 08:54:57.938258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/usr/local/cuda/lib
2022-04-17 08:54:57.939085: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)
# finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

if dataset == "valid":
    # [380:430]
    exp_dataset = valid_dataset.load_dataset(valid_path).head(100)
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
# exp_dataset = dd.from_pandas(exp_dataset, npartitions=10)
# exp_dataset = te.DataFrame.from_pandas(exp_dataset)
text,label = None,None

In [17]:
if downstream_task == 'sentiment':
    text = 'text'
    label = 'sentiment'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.text,
            true_label = row.sentiment,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.tweet.values,
            true_label = row.label.values,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )

before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_semantic_sim'] = exp_dataset["perturbed_semantic_sim"].mean()
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()

100%|█████████████████████████████████████████████████████████████| 100/100 [03:11<00:00,  1.91s/it]


In [19]:
exp_dataset.head(20)

,text,sentiment,perturbed_text,perturbed_semantic_sim,pred_label,pred_proba,perturbed_label,perturbed_proba,translated_word(s),running_time(s),before_attack_acc,after_attack_acc,avg_semantic_sim,avg_running_time(s)
0,"meski masa kampanye sudah selesai , bukan bera...",1,meski masa kampanye sudah selesai bukan mean h...,0.753872,1,"[0.0365, 0.8362, 0.1273]",0,"[0.4917, 0.4487, 0.0596]","{'upaya': 'effort', 'berati': 'mean', 'mengere...",0.7329,0.96,0.87,0.851678,1.910111
1,tidak enak,2,tidak nice,0.540959,2,"[0.0011, 0.001, 0.9979]",2,"[0.0019, 0.001, 0.9971]",{'enak': 'nice'},0.6008,NaN,NaN,NaN,NaN
2,restoran ini menawarkan makanan sunda . kami m...,0,restaurant ini menawarkan food sunda kami meme...,0.823257,0,"[0.9976, 0.0016, 0.0008]",0,"[0.9654, 0.0226, 0.0121]","{'enak': 'nice', 'makanan': 'food', 'variatif'...",4.2013,NaN,NaN,NaN,NaN
3,lokasi di alun alun masakan padang ini cukup t...,0,location di alun alun masakan field ini cukup ...,0.862981,0,"[0.9982, 0.0012, 0.0005]",0,"[0.9933, 0.006, 0.0007]","{'mengenyangkan': 'filling', 'gule': 'gule', '...",2.5275,NaN,NaN,NaN,NaN
4,betapa bejad kader gerindra yang anggota dprd ...,2,betapa depraved cadre gerindra yang anggota dp...,0.824325,2,"[0.0045, 0.0029, 0.9926]",2,"[0.1607, 0.1407, 0.6986]","{'bejad': 'depraved', 'partai': 'party', 'menc...",1.3139,NaN,NaN,NaN,NaN
5,kampiun bistro berada di jalan . kebon kawung ...,0,champion bistro berada di jalan kebon kawung a...,0.889336,0,"[0.9971, 0.0025, 0.0004]",0,"[0.9969, 0.0026, 0.0005]","{'musik': 'music', 'nya': 'his', 'bandung': 'B...",2.3597,NaN,NaN,NaN,NaN
6,kamar nya sempit tidak ada tempat menyimpan ba...,2,room nya narrow tidak ada tempat menyimpan goo...,0.912295,2,"[0.0098, 0.0017, 0.9886]",2,"[0.1308, 0.007, 0.8622]","{'sempit': 'narrow', 'kamar': 'room', 'uchiwa'...",1.4899,NaN,NaN,NaN,NaN
7,"saya tadi mampir ke restauran ini , mungkin le...",0,saya tadi mampir ke restaurant ini mungkin let...,0.904444,0,"[0.9983, 0.0013, 0.0004]",0,"[0.9958, 0.003, 0.0012]","{'suka': 'like', 'enak': 'nice', 'super': 'sup...",3.2132,NaN,NaN,NaN,NaN
8,jangan percaya dengan agama buddha . agama itu...,2,jangan percaya dengan agama buddha agama itu s...,0.939600,2,"[0.0007, 0.0011, 0.9981]",2,"[0.0012, 0.0016, 0.9972]","{'sesat': 'perverted', 'tuhan': 'lord'}",0.3933,NaN,NaN,NaN,NaN
9,kafe ini menawarkan sesuatu hal yang sebelumny...,0,kafe ini menawarkan sesuatu hal yang sebelumny...,0.930858,0,"[0.9965, 0.0024, 0.0011]",0,"[0.9968, 0.0015, 0.0017]","{'lezat': 'delicious', 'goreng': 'fry', 'enak'...",1.4795,NaN,NaN,NaN,NaN


### After Logit Improvement

In [263]:
attack(
    text_ls = exp_dataset.iloc[[7]].text.item(),
    true_label = exp_dataset.iloc[[7]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

ic| original_text: 'lucu'
ic| end_time_logit: 0.0119
ic| end_time_arrops: 0.0011
ic| running_time_a: 0.1544
ic| top_words_perturb: [(0, 'lucu', 0.6613012860584355)]
ic| end_time_pert: 0.7008
ic| end_time_use: 0.0055
ic| running_time: 0.9818


('funny',
 0.2863596,
 0,
 array([0.9945, 0.0015, 0.004 ]),
 0,
 array([0.9941, 0.0045, 0.0014]),
 {'lucu': 'funny'},
 0.9818)

### Coba optimize di USE nya

In [16]:
attack(
    text_ls = exp_dataset.iloc[[0]].text.item(),
    true_label = exp_dataset.iloc[[0]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

('meski masa kampanye sudah selesai bukan mean habis pula effort hoist tingkat kedipilihan elektabilitas',
 0.7538721,
 1,
 array([0.0365, 0.8362, 0.1273]),
 0,
 array([0.4917, 0.4487, 0.0596]),
 {'upaya': 'effort', 'berati': 'mean', 'mengerek': 'hoist'},
 1.0341)

In [10]:
def logit_prob(text_ls, predictor, tokenizer):\
    subwords = tokenizer.encode(text_ls)
    subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)

    logits = predictor(subwords)[0]
    orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    
    orig_probs = F.softmax(logits, dim=-1).squeeze()
    orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
    
    return orig_label, orig_probs, orig_prob

t = 'meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .'
logit_prob(t, finetuned_model, tokenizer)

(1,
 tensor([0.0365, 0.8362, 0.1273], device='cuda:0', grad_fn=<SqueezeBackward0>),
 array(0.8362002, dtype=float32))

### Coba Optimize step 2+codemix perturbation pake batch

In [ ]:
attack(
    text_ls = exp_dataset.iloc[[48]].text.item(),
    true_label = 0,
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

In [283]:
exp_dataset.to_csv('a.csv')

In [ ]:
def codemix_perturbation(words, target_lang, words_perturb):
    """
    'su': 'sundanese'
    'jw': 'javanese'
    'ms': 'malay'
    'en': 'english'
    """
    translator = GoogleTranslator(source="id", target=target_lang)
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
        
    new_wp_trans = dict(zip(new_wp, translator.translate_batch(new_wp)))
    
    supported_langs = ["su", "jw", "ms", "en", "fr", "it"]
    
    if target_lang not in supported_langs:
        raise ValueError('Language Unavailable')
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word] if word == perturb_word and word.isalpha() else word for word in new_words]

    return ' '.join(new_words), new_wp_trans

def synonym_replacement(words, words_perturb):    
    # new_words = words.copy()
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, get_synonyms(new_wp[0])[0]))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
    
    wp_trans=[]
    for wp in new_wp:
        wp_trans.append(get_synonyms(wp)[0])
        print(wp,wp_trans)
    
    new_wp_trans = dict(zip(new_wp, wp_trans))
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word[1]] if word == perturb_word[1] and word.isalpha() else word for word in new_words]

    sentence = ' '.join(new_words)
    
    return sentence, new_wp_trans

In [ ]:
word = 'buat kalian yang cuma tahu tempat makan malam romantis yang ada di dago , boleh dicoba deh makan di burgundy sini , di sini sih menurut saya tempat yang enak banget buat ngadem selain tempat nya yang hening kita juga bisa melihat rerumputan yang hijau meski cuma di taman nya , hehehe . tapi balik lagi buat saya sih makanan , haha . tidak usah diragukan .'
words = word.split()
wp = [(59, 'diragukan', 0.001480978060399707), (27, 'enak', 0.0006103660272316347), (18, 'burgundy', 0.0005351185233486433), (55, 'makanan', 0.000532135112141674), (12, 'dago', 0.0004499789485095107), (8, 'romantis', 0.0003965279976370084), (46, 'taman', 0.00039581217539819136), (40, 'rerumputan', 0.00039318750260974866), (7, 'malam', 0.000384060906586825)]
target_lang = "en"

start_time = time.time()
print(synonym_replacement(words, wp))
print(round(time.time() - start_time, 2))

In [ ]:
mbert-sentiment-codemixing-jw-adv-0.6-valid
# 279/1260
850/1260
new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
IndexError: list index out of range

In [ ]:
attack(
    text_ls = exp_dataset.iloc[[5]].text.item(),
    true_label = exp_dataset.iloc[[5]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = 0.6,
    perturbation_technique = perturbation_technique,
    lang_codemix = "jw",
    sim_predictor = use
)

In [ ]:
exp_dataset.iloc[[7]]

In [ ]:

import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
# !pip list cudnn

In [ ]:
texts = ['diragukan', 'enak', 'burgundy', 'makanan', 'dago', 'romantis', 'taman', 'rerumputan', 'malam']

start_time = time.time()
translated_1 = []
translator = GoogleTranslator(source="id", target="en")
for txt in texts:
    translated_1.append(translator.translate(txt))
print(round(time.time() - start_time, 2))

In [ ]:
start_time = time.time()
translated_2 = translator.translate_batch(texts)
print(round(time.time() - start_time, 2))

In [ ]:
# !. myenv/bin/activate
# !pip uninstall tensorflow -y
# !pip uninstall tensorflow-gpu -y

!pip install tensorflow --ignore-installed

In [83]:
import transformers
print(transformers.__version__)

4.17.0


In [222]:
def logit_prob(text_ls, predictor, tokenizer):
    subwords = tokenizer.encode(text_ls)
    subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)

    logits = predictor(subwords)[0]
    orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    
    orig_probs = F.softmax(logits, dim=-1).squeeze()
    orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
    
    return orig_label, orig_probs, orig_prob

In [ ]:
# %export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!echo $LD_LIBRARY_PATH

In [33]:
def logit_prob(text_ls, predictor, tokenizer):
    with torch.inference_mode():
        subwords = np.array(tokenizer(text_ls)['input_ids'])
        # ic(subwords)
        if subwords.ndim <= 1:
            subwords = torch.tensor(subwords).view(1, -1).to(predictor.device)
            logits = predictor(subwords)[0]
            orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
        else:
            subwords = torch.tensor(subwords).to(predictor.device)
            logits = finetuned_model(subwords)[0]
            orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
    
    return orig_label, orig_probs, orig_prob

In [ ]:
subwords = np.array([
    [2, 4, 4700, 500, 5622, 7884, 3],
    [2, 422, 4, 500, 5622, 7884, 3],
    [2, 422, 4700, 4, 5622, 7884, 3],
    [2, 422, 4700, 500, 4, 7884, 3],
    [2, 422, 4700, 500, 5622, 4, 3]
])
subwords = torch.tensor(subwords).to(finetuned_model.device)
subwords

tensor([[   2,    4, 4700,  500, 5622, 7884,    3],
        [   2,  422,    4,  500, 5622, 7884,    3],
        [   2,  422, 4700,    4, 5622, 7884,    3],
        [   2,  422, 4700,  500,    4, 7884,    3],
        [   2,  422, 4700,  500, 5622,    4,    3]], device='cuda:0')

In [15]:
subwords = np.array([2, 4, 4700, 500, 5622, 7884, 3])
print(subwords.ndim)
subwords = torch.tensor(subwords).view(1,-1).to(finetuned_model.device)

1


In [17]:
subwords

tensor([[   2,    4, 4700,  500, 5622, 7884,    3]], device='cuda:0')

In [219]:
text_ls = 'pemancing nanti kasus nya kek anggota fpi ikut membongkar obattan malah ditangkap'.split(' ')
# text_ls = 'mau marah sama koneksi indosat'.split(' ')
text_ls = 'lucu'.split(' ')
len_text = len(text_ls)
leave_1_texts = [' '.join(text_ls[:ii] + [tokenizer.mask_token] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]

In [220]:
leave_1_texts

['[MASK]']

In [200]:
start_time_b = time.time()
# ic(tokenizer(leave_1_texts)['input_ids'])
subwords = tokenizer.batch_encode_plus(leave_1_texts, return_tensors="pt", padding=True)['input_ids'].to(finetuned_model.device)
# ic(subwords.shape)
# subwords = np.array([
#     [2, 4, 4700, 500, 5622, 7884, 3],
#     [2, 422, 4, 500, 5622, 7884, 3],
#     [2, 422, 4700, 4, 5622, 7884, 3],
#     [2, 422, 4700, 500, 4, 7884, 3],
#     [2, 422, 4700, 500, 5622, 4, 3]
# ])
# subwords = torch.tensor(subwords).to(finetuned_model.device)
# ic(subwords)
logits = finetuned_model(subwords).logits
# ic(logits)
# log = finetuned_model(subwords)
# ic(log)
orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
# ic(orig_label)
orig_probs = F.softmax(logits, dim=-1).squeeze()
# ic(orig_probs)
orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
# ic(orig_prob)
end_time_b = round(time.time() - start_time_b, 4)
ic(end_time_b)

ic| end_time_b: 0.0258


0.0258

In [223]:
def logit_prob(text_ls, predictor, tokenizer):
    # original_text = text_ls
    # print(text_ls)
    subwords = tokenizer.encode(text_ls)
    subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)
    logits = predictor(subwords)[0]
    orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    
    orig_probs = F.softmax(logits, dim=-1).squeeze()
    orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
    
    return orig_label, orig_probs, orig_prob, subwords

start_time_b = time.time()

orig_probs_leave_1_ext=[]
orig_prob_leave_1_ext=[]
orig_label_leave_1_ext=[]
subwords_ls = []
for text_leave_1 in leave_1_texts:
    orig_label_leave_1, orig_probs_leave_1, orig_prob_leave_1,subwords = logit_prob(text_leave_1, finetuned_model, tokenizer)
    orig_probs_leave_1_ext.append(orig_probs_leave_1.detach().cpu().numpy())
    orig_prob_leave_1_ext.append(orig_prob_leave_1)
    orig_label_leave_1_ext.append(orig_label_leave_1)
    ic(len(text_leave_1))
    ic(text_leave_1)
    ic(len(subwords[0]))
    subwords_ls.append(subwords)

end_time_b = round(time.time() - start_time_b, 4)
ic(end_time_b)

ic(orig_probs_leave_1_ext)
ic(orig_label_leave_1_ext)
ic(orig_prob_leave_1_ext)
ic(subwords_ls)

ic| len(text_leave_1): 6
ic| text_leave_1: '[MASK]'
ic| len(subwords[0]): 3
ic| end_time_b: 0.0698
ic| orig_probs_leave_1_ext: [array([0.5398942 , 0.04156226, 0.41854346], dtype=float32)]
ic| orig_label_leave_1_ext: [0]
ic| orig_prob_leave_1_ext: [array(0.5398942, dtype=float32)]
ic| subwords_ls: [tensor([[2, 4, 3]], device='cuda:0')]


[tensor([[2, 4, 3]], device='cuda:0')]

In [ ]:
# orig_label
# orig_probs = F.softmax(logits, dim=-1).squeeze()
# orig_probs
for idx, label in enumerate(orig_label):
    print(label)
    print(F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy())
orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
# orig_prob = F.softmax(logits, dim=-1).squeeze()
print(orig_prob)

In [232]:
t = 'mau marah sama koneksi indosat'
t.split()

['mau', 'marah', 'sama', 'koneksi', 'indosat']

In [ ]:
if len(text_ls) <= 1:
        if perturbation_technique == "codemixing":
            top_words_perturb = (0, text_ls, 1.000)
            perturbed_text,translated_words = codemix_perturbation(text_ls, lang_codemix, top_words_perturb)
        elif perturbation_technique == "synonym_replacement":
            perturbed_text,translated_words = synonym_replacement(text_ls, top_words_perturb)
            
        perturb_sim_score = sim_predictor.semantic_sim(original_text, perturbed_text)
        
        if perturb_sim_score < sim_score_threshold:
            running_time = round(time.time() - start_time, 2)
            orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
            
            running_time = round(time.time() - start_time, 2)
            return original_text, 1.000, orig_label, orig_probs, orig_label, orig_probs,'', running_time
        else:
            running_time = round(time.time() - start_time, 2)
            perturbed_label, perturbed_probs, perturbed_prob = logit_prob(text_ls, predictor, tokenizer)
            orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
            
            running_time = round(time.time() - start_time, 2)
            return perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time
